In [36]:
import pandas as pd

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

#this gives us phrases
from gensim.models.phrases import Phrases, Phraser

from ast import literal_eval

In [3]:
ls -lh data

 Volume in drive C has no label.
 Volume Serial Number is 16EE-C352

 Directory of C:\Users\user


 Directory of C:\Users\user\data

01/10/2019  14:01    <DIR>          .
01/10/2019  14:01    <DIR>          ..
16/09/2019  14:01       133,415,034 job_ofer.csv
12/09/2019  23:04       240,342,787 reviews_data.txt
               2 File(s)    373,757,821 bytes
               2 Dir(s)  63,119,589,376 bytes free


File Not Found


In [4]:
df = pd.read_csv('data/job_ofer.csv')
df.shape

(36109, 8)

In [5]:
df.head()

,title,company_name,address,description,seniority_level,employment_type,job_function,industries
0,Machine Learning Engineer,Intellipro Group Inc,"Palo Alto, CA, US","['About The Company', ""W*** is reshaping the f...",Entry level,Full-time,Engineering,Information Technology and Services
1,Deep Learning Applied Researcher - Chicago,Ethosia,"Chicago, IL, US","['תיאור המשרה', 'Deep learning for Computer Vi...",Associate,Full-time,Other,Information Technology and Services
2,Machine Learning Engineer,Motorola Solutions,"Chicago, IL, US","['Company Overview', 'At Motorola Solutions, w...",Entry level,Full-time,Engineering,Information Technology and Services
3,Machine Learning / Data Scientist,Proprius LLC,"San Francisco, CA, US",['Our client is a digital invention agency foc...,Entry level,Full-time,Engineering,Information Technology and Services
4,Cloud Architect,TCS,"Framingham, Massachusetts, United States","['Technical/Functional Skills', ' ', 'Good to ...",Mid-Senior level,Full-time,Engineering,Information Technology and Services


# Word2Vec

In [6]:
corpus = df['title'].map( simple_preprocess )

In [7]:
model = Word2Vec(corpus, size=100, window=2, min_count=1)

In [8]:
model.wv.most_similar('machine')

[('deep', 0.9546080827713013),
 ('captivate', 0.8814845085144043),
 ('edge', 0.880230188369751),
 ('inference', 0.8531221151351929),
 ('hux', 0.8491808176040649),
 ('natural', 0.8458027839660645),
 ('computer', 0.8425424098968506),
 ('applied', 0.8341013789176941),
 ('nlp', 0.8311324119567871),
 ('big', 0.8294161558151245)]

# Example

In [9]:
corpus = [
    ['machine', 'learning', 'c'],
    ['machine', 'learning', 'x'],
    ['y', 'machine', 'learning', 'w'],
    ['q', 'machine', 'learning', 'u', 'k'],
]
bigram = Phraser( Phrases(corpus, min_count=1, threshold=1) )

#We connect macine and learning into  1 word machine_learning
bigram[ ['machine', 'learning', 'c'] ]

['machine_learning', 'c']

# Title + phrases

In [10]:
#For one sentence
title_corpus = df['title'].map( simple_preprocess )

title_bigram = Phraser( Phrases(title_corpus, min_count=1, threshold=1) )

In [11]:
title_bigram[simple_preprocess('Machine Learning Engineer')]

['machine_learning', 'engineer']

In [12]:
title_corpus_phrase = [ title_bigram[sent] for sent in title_corpus ]
model = Word2Vec(title_corpus_phrase, size=100, window=2, min_count=1)

In [13]:
model.wv.most_similar('machine')

[('it_sr', 0.9582561254501343),
 ('assistant_general', 0.9555681943893433),
 ('night', 0.9554369449615479),
 ('product_insights', 0.9544827938079834),
 ('ft_am', 0.9541812539100647),
 ('engineer_ios', 0.9533193111419678),
 ('millwright', 0.9531430006027222),
 ('automotive_guest', 0.9524208903312683),
 ('berlin_nj', 0.9523929953575134),
 ('midwest', 0.9523296356201172)]

In [14]:
def prepare_corpus(corpus, bigram):
    for sent in corpus:
        yield bigram[sent] + sent
    

In [15]:
ext_corp = list(prepare_corpus(title_corpus, title_bigram))
title_model = Word2Vec( ext_corp, size=100, window=2, min_count=1 )

In [17]:
title_model.wv.most_similar('machine_learning')

[('computer_vision', 0.9892956018447876),
 ('deep_learning', 0.9813571572303772),
 ('learning', 0.9808920621871948),
 ('deep', 0.9798362255096436),
 ('rockville', 0.9718903303146362),
 ('nlp', 0.9709010720252991),
 ('artificial_intelligence', 0.9676969051361084),
 ('computer', 0.966477632522583),
 ('data_engineer', 0.9654456377029419),
 ('ad_tech', 0.9651151895523071)]

# Description

In [32]:
descr_corpus = df['description'].map( simple_preprocess )

descr_bigram = Phraser( Phrases(descr_corpus, min_count=2, threshold=1) )

In [33]:
#simple_preprocess( df.sample()['description'].values[0] )
#ext_descr_corp = list(prepare_corpus(descr_corpus, descr_bigram))
descr_model = Word2Vec( descr_corpus, size=100, window=2, min_count=1 )

In [34]:
descr_model.wv.most_similar('keras')

[('pytorch', 0.9520508646965027),
 ('tensorflow', 0.9489986896514893),
 ('theano', 0.9453537464141846),
 ('caffe', 0.9411250352859497),
 ('scipy', 0.9253391027450562),
 ('mxnet', 0.916308581829071),
 ('numpy', 0.9135805368423462),
 ('matplotlib', 0.8863964676856995),
 ('jupyter', 0.8830247521400452),
 ('sklearn', 0.8815372586250305)]

In [35]:
title_model.wv.most_similar('keras')

[('massage_therapist', 0.99440598487854),
 ('bristol_oxford', 0.9940013289451599),
 ('des', 0.9939993619918823),
 ('consultant_sap', 0.9937916994094849),
 ('de', 0.9934953451156616),
 ('staff_nurse', 0.9934335947036743),
 ('nurse_lpn', 0.9933348894119263),
 ('hrs_per', 0.9932541847229004),
 ('deer', 0.9932483434677124),
 ('six_sigma', 0.9932183027267456)]

In [39]:
for line in df.sample()['description'].map(literal_eval).values[0]:
    print(line)
    print("")

What We Do

NFF works toward a more just and vibrant society through:

Financing 

that helps nonprofits and social enterprises expand opportunity in low- and middle-income communities.

Consulting 

that helps nonprofits and funders connect money to mission and community impact.

Partnering 

with service providers, funders, and investors to turn money and goodwill into positive outcomes.

Learning 

that shares experiences and expertise – from ideas to anecdotes to cutting-edge data and analysis.

Nonprofit Finance Fund® (NFF®) advances missions and social progress in underserved communities through financing, consulting, partnerships, and knowledge-sharing. A leading Community Development Financial Institution (CDFI), NFF currently manages over $329 million. Since 1980, we have provided $871 million in financing and access to additional capital in support of over $2.9 billion in projects for thousands of organizations nationwide.

Position Summary

The Advisory Services team is seek